In [1]:
import os
import json
import time
import pandas as pd
import numpy as np
from pathlib import Path
from openai import OpenAI
import re
import glob
from bs4 import BeautifulSoup
from tqdm import tqdm
import warnings
warnings.filterwarnings('ignore')

# 设置API密钥
os.environ["DASHSCOPE_API_KEY"] = "sk-2ea9416b45e04af6b6aa72d3c2ade52f"

# 初始化OpenAI客户端，使用阿里云百炼平台
client = OpenAI(
    api_key=os.getenv("DASHSCOPE_API_KEY"),
    base_url="https://dashscope.aliyuncs.com/compatible-mode/v1"
)

# 设置环境变量和配置
os.environ["TOKENIZERS_PARALLELISM"] = "false"  # 避免警告
OUTPUT_DIR = "生成结果/last_quadrant"
os.makedirs(OUTPUT_DIR, exist_ok=True)

# 数据文件
data_files = {
    "excel_file_1": "生成结果/last_quadrant/四象限数据-痛点.xlsx",
    "excel_file_2": "生成结果/last_quadrant/四象限数据-需求.xlsx",
    "excel_file_3": "生成结果/last_quadrant/四象限数据-场景匹配.xlsx",
    "excel_file_4": "生成结果/social_media/Comment-模型结果.xlsx",
    "excel_file_5": "生成结果/social_media/Like-模型结果.xlsx",
    "excel_file_6": "生成结果/social_media/Repost-模型结果.xlsx",
    "excel_file_7": "生成结果/social_media/Share-模型结果.xlsx",
    "excel_file_8": "生成结果/social_media/View-模型结果.xlsx",
    "md_file_1": "生成结果/social_media/理论假设.md"
}

# 自动搜索三个文件夹中的txt文件
defect_dir = "生成结果/defect_analysis/"
needs_dir = "生成结果/needs_analysis/"
matches_dir = "生成结果/matches_analysis/"

# 在每个文件夹中查找第一个txt文件
def find_first_txt(directory):
    txt_files = glob.glob(os.path.join(directory, "*.txt"))
    return txt_files[0] if txt_files else None

# 更新data_files字典
defect_txt = find_first_txt(defect_dir)
if defect_txt:
    data_files["txt_file_1"] = defect_txt
    print(f"txt_file_1: {defect_txt}")
else:
    print(f"警告: 在 {defect_dir} 中未找到txt文件")

needs_txt = find_first_txt(needs_dir)
if needs_txt:
    data_files["txt_file_2"] = needs_txt
    print(f"txt_file_2: {needs_txt}")
else:
    print(f"警告: 在 {needs_dir} 中未找到txt文件")

matches_txt = find_first_txt(matches_dir)
if matches_txt:
    data_files["txt_file_3"] = matches_txt
    print(f"txt_file_3: {matches_txt}")
else:
    print(f"警告: 在 {matches_dir} 中未找到txt文件")

# Prompt文件路径列表
prompt_files = {
    "prompt_1": "Data/建议表1-表达-流量关系表Prompt.txt",
    "prompt_2": "Data/建议表2-消费者旅程不同阶段的产品表达关注表Prompt.txt",
    "prompt_3": "Data/建议表3-品类功能利益点情感利益点分析表Prompt.txt",
    "prompt_4": "Data/建议表4-产品改进建议Prompt.txt",
    "prompt_5": "Data/建议表5-情感利益点改进建议Prompt.txt",
    "prompt_6": "Data/建议表6-改进优先级排序Prompt.txt",
}

# 输出文件名映射
output_file_names = {
    1: "建议表1-表达-流量关系表.xlsx",
    2: "建议表2-消费者旅程不同阶段的产品表达关注表.xlsx",
    3: "建议表3-品类功能利益点情感利益点分析表.xlsx",
    4: "建议表4-产品改进建议.xlsx",
    5: "建议表5-情感利益点改进建议.xlsx",
    6: "建议表6-改进优先级排序.xlsx"
}

# 读取文件内容的函数
def read_file_content(file_path):
    """读取文件内容，根据文件扩展名处理不同类型的文件"""
    if not os.path.exists(file_path):
        return f"文件不存在: {file_path}"
    
    file_ext = os.path.splitext(file_path)[1].lower()
    
    try:
        if file_ext == '.xlsx' or file_ext == '.xls':
            # 读取Excel文件
            dfs = {}
            xl = pd.ExcelFile(file_path)
            for sheet_name in xl.sheet_names:
                df = pd.read_excel(file_path, sheet_name=sheet_name)
                dfs[sheet_name] = df.to_string()
            
            # 处理只有一个工作表的情况
            if len(dfs) == 1:
                return f"Excel文件 {os.path.basename(file_path)} 内容:\n{list(dfs.values())[0]}"
            else:
                content = f"Excel文件 {os.path.basename(file_path)} 内容:\n"
                for sheet_name, sheet_content in dfs.items():
                    content += f"\n工作表: {sheet_name}\n{sheet_content}\n"
                return content
        
        elif file_ext == '.md':
            # 读取Markdown文件
            with open(file_path, 'r', encoding='utf-8') as file:
                return f"Markdown文件 {os.path.basename(file_path)} 内容:\n{file.read()}"
        
        elif file_ext == '.txt':
            # 读取文本文件
            with open(file_path, 'r', encoding='utf-8') as file:
                return f"文本文件 {os.path.basename(file_path)} 内容:\n{file.read()}"
        
        else:
            return f"不支持的文件类型: {file_path}"
    
    except Exception as e:
        return f"读取文件 {file_path} 时出错: {str(e)}"

# 提取HTML内容的函数
def extract_html_from_response(response_text):
    """从API响应中提取HTML代码块"""
    html_pattern = r"```html\s*([\s\S]*?)\s*```"
    match = re.search(html_pattern, response_text)
    
    if match:
        html_content = match.group(1).strip()
        return html_content
    
    # 如果没有明确的HTML标记，尝试提取任何看起来像HTML的内容
    if "<html" in response_text and "</html>" in response_text:
        html_start = response_text.find("<html")
        html_end = response_text.find("</html>") + 7
        return response_text[html_start:html_end]
    
    # 尝试提取表格内容
    if "<table" in response_text and "</table>" in response_text:
        table_pattern = r"(<table[\s\S]*?</table>)"
        tables = re.findall(table_pattern, response_text)
        if tables:
            return "<html><body>" + "".join(tables) + "</body></html>"
    
    return None

# 将HTML表格转换为DataFrame并保存为Excel的函数
def html_to_excel(html_content, output_path):
    """将HTML表格内容转换为DataFrame并保存为Excel文件"""
    try:
        # 解析HTML
        soup = BeautifulSoup(html_content, 'html.parser')
        tables = soup.find_all('table')
        
        if not tables:
            print(f"警告: 在HTML内容中未找到表格")
            # 保存原始HTML内容以供参考
            html_path = output_path.replace('.xlsx', '.html')
            with open(html_path, 'w', encoding='utf-8') as f:
                f.write(html_content)
            print(f"已保存原始HTML到: {html_path}")
            return False
        
        # 创建Excel写入器
        with pd.ExcelWriter(output_path, engine='openpyxl') as writer:
            # 处理每个表格
            for i, table in enumerate(tables):
                # 尝试提取表格标题
                title = None
                prev = table.find_previous(['h1', 'h2', 'h3', 'h4', 'h5', 'h6', 'caption', 'p'])
                if prev and prev.name != 'table':
                    title = prev.get_text().strip()
                
                # 设置工作表名
                sheet_name = f"表格{i+1}"
                if title:
                    # 清理标题以用作工作表名
                    clean_title = re.sub(r'[\\/*?:\[\]]', '', title)
                    if len(clean_title) > 0:
                        sheet_name = clean_title[:30]  # Excel工作表名长度限制
                
                # 提取表头
                headers = []
                header_row = table.find('tr')
                if header_row:
                    for th in header_row.find_all(['th', 'td']):
                        headers.append(th.get_text().strip())
                
                # 提取数据行
                rows = []
                data_rows = table.find_all('tr')[1:] if headers else table.find_all('tr')
                for row in data_rows:
                    cells = row.find_all(['td', 'th'])
                    if cells:
                        row_data = [cell.get_text().strip() for cell in cells]
                        rows.append(row_data)
                
                # 创建DataFrame
                if headers and len(headers) == len(rows[0]) if rows else False:
                    df = pd.DataFrame(rows, columns=headers)
                else:
                    # 如果表头有问题，使用默认列名
                    df = pd.DataFrame(rows)
                
                # 保存到Excel工作表
                df.to_excel(writer, sheet_name=sheet_name, index=False)
            
            print(f"成功将{len(tables)}个表格保存到Excel: {output_path}")
            return True
    
    except Exception as e:
        print(f"处理HTML表格时出错: {str(e)}")
        # 保存HTML以供调试
        html_path = output_path.replace('.xlsx', '.html')
        with open(html_path, 'w', encoding='utf-8') as f:
            f.write(html_content)
        print(f"已保存原始HTML到: {html_path}")
        return False

# 保存文本响应内容为Excel的函数
def text_to_excel(response_text, output_path):
    """如果无法提取HTML表格，尝试直接从文本内容中提取表格信息"""
    try:
        # 尝试查找表格式文本内容 (通常以 | 分隔的文本)
        table_pattern = r"(\|[^\n]+\|\n\|[-:| ]+\|\n(?:\|[^\n]+\|\n)+)"
        table_matches = re.findall(table_pattern, response_text)
        
        if table_matches:
            with pd.ExcelWriter(output_path, engine='openpyxl') as writer:
                for i, table_text in enumerate(table_matches):
                    # 解析表格行
                    rows = table_text.strip().split('\n')
                    
                    # 提取表头
                    header_row = rows[0]
                    headers = [h.strip() for h in header_row.split('|')[1:-1]]
                    
                    # 跳过分隔行
                    data_rows = rows[2:]
                    
                    # 提取数据
                    data = []
                    for row in data_rows:
                        cells = [cell.strip() for cell in row.split('|')[1:-1]]
                        if cells:
                            data.append(cells)
                    
                    # 创建DataFrame
                    df = pd.DataFrame(data, columns=headers)
                    
                    # 保存到Excel
                    df.to_excel(writer, sheet_name=f"表格{i+1}", index=False)
            
            print(f"成功从文本中提取{len(table_matches)}个表格到Excel: {output_path}")
            return True
        else:
            # 如果找不到表格式文本，保存原始响应
            txt_path = output_path.replace('.xlsx', '.txt')
            with open(txt_path, 'w', encoding='utf-8') as f:
                f.write(response_text)
            print(f"未找到表格式内容，已保存原始响应到: {txt_path}")
            return False
    
    except Exception as e:
        print(f"处理文本内容时出错: {str(e)}")
        # 保存原始响应
        txt_path = output_path.replace('.xlsx', '.txt')
        with open(txt_path, 'w', encoding='utf-8') as f:
            f.write(response_text)
        print(f"已保存原始响应到: {txt_path}")
        return False

# 主函数：逐个处理prompt和输出Excel
def process_each_prompt():
    # 首先读取所有数据文件的内容
    print("读取所有数据文件...")
    data_content = {}
    for key, file_path in data_files.items():
        if os.path.exists(file_path):
            data_content[key] = read_file_content(file_path)
            print(f"✓ 已读取: {key}")
        else:
            print(f"✗ 文件不存在: {file_path}")
    
    print(f"\n所有数据文件读取完成，共 {len(data_content)} 个文件\n")
    
    # 逐个处理每个prompt
    for prompt_num in range(1, 7):
        prompt_key = f"prompt_{prompt_num}"
        prompt_path = prompt_files[prompt_key]
        
        # 使用新的命名方式
        output_file_name = output_file_names[prompt_num]
        output_path = os.path.join(OUTPUT_DIR, output_file_name)
        
        print(f"\n{'='*60}")
        print(f"处理 Prompt {prompt_num}: {os.path.basename(prompt_path)}")
        print(f"输出文件: {output_file_name}")
        print(f"{'='*60}")
        
        try:
            # 读取prompt内容
            with open(prompt_path, 'r', encoding='utf-8') as f:
                prompt_content = f.read()
            
            print(f"Prompt长度: {len(prompt_content)} 字符")
            
            # 构建完整的提示内容
            full_prompt = f"""
我需要你根据以下prompt和数据生成一个Excel表格。
请仔细阅读prompt要求，并根据提供的数据生成符合要求的表格。
请以HTML表格格式输出，我会将其转换为Excel。

## PROMPT 要求:
{prompt_content}

## 数据内容:
"""
            # 添加所有数据文件内容
            for key in data_content:
                full_prompt += f"--- {key} ---\n{data_content[key]}\n\n"
            
            full_prompt += "\n请提供完整的HTML表格，确保表格格式规范，可以直接转换为Excel。不需要其他解释，只需要给出表格的HTML代码，并用```html 和 ``` 包裹。"
            
            # 调用API
            print(f"调用API中...")
            start_time = time.time()
            
            try:
                # 使用qwq-plus模型调用API，启用流式输出
                print("\n" + "=" * 20 + "思考过程" + "=" * 20 + "\n")
                stream = client.chat.completions.create(
                    model="qwq-plus",  # 使用qwq-plus模型
                    messages=[
                        {'role': 'user', 'content': full_prompt}
                    ],
                    stream=True  # 启用流式输出 - QwQ模型必须启用
                )
                
                # 处理流式响应
                reasoning_content = ""  # 记录思考过程
                result_content = ""     # 记录最终结果
                is_answering = False    # 标记是否开始回答
                
                for chunk in stream:
                    # 如果chunk.choices为空，打印usage
                    if not chunk.choices:
                        if hasattr(chunk, 'usage'):
                            print("\nAPI使用情况:")
                            print(chunk.usage)
                    else:
                        delta = chunk.choices[0].delta
                        
                        # 处理思考过程
                        if hasattr(delta, 'reasoning_content') and delta.reasoning_content is not None:
                            print(delta.reasoning_content, end='', flush=True)
                            reasoning_content += delta.reasoning_content
                        
                        # 处理回答内容
                        if hasattr(delta, 'content') and delta.content is not None:
                            # 标记开始回答
                            if delta.content != "" and not is_answering:
                                print("\n\n" + "=" * 20 + "回答内容" + "=" * 20 + "\n")
                                is_answering = True
                            
                            # 累积回答内容
                            result_content += delta.content
                            print(delta.content, end='', flush=True)
                
                end_time = time.time()
                print("\n\nAPI调用完成，耗时: {:.2f}秒".format(end_time - start_time))
                
                # 提取文件名的主要部分用于中间文件命名
                base_name = os.path.splitext(output_file_name)[0]
                
                # 保存思考过程
                if reasoning_content:
                    reasoning_path = os.path.join(OUTPUT_DIR, f"{base_name}_reasoning.txt")
                    with open(reasoning_path, 'w', encoding='utf-8') as f:
                        f.write(reasoning_content)
                    print(f"已保存思考过程到: {reasoning_path}")
                
                # 保存原始回答
                result_path = os.path.join(OUTPUT_DIR, f"{base_name}_result.txt")
                with open(result_path, 'w', encoding='utf-8') as f:
                    f.write(result_content)
                print(f"已保存原始回答到: {result_path}")
                
                # 处理回答内容并导出Excel
                print(f"处理API响应并生成Excel...")
                
                # 首先尝试提取HTML内容
                html_content = extract_html_from_response(result_content)
                
                if html_content:
                    # 如果找到HTML内容，转换为Excel
                    success = html_to_excel(html_content, output_path)
                    if success:
                        print(f"✓ 成功生成Excel文件: {output_path}")
                    else:
                        print(f"⚠ 无法从HTML生成Excel文件")
                else:
                    # 如果没有HTML内容，尝试从文本中提取表格
                    print("未找到HTML内容，尝试从文本中提取表格...")
                    success = text_to_excel(result_content, output_path)
                    
                    if success:
                        print(f"✓ 成功从文本生成Excel文件: {output_path}")
                    else:
                        print(f"⚠ 无法生成Excel文件，请查看保存的原始响应")
                
            except Exception as e:
                print(f"API调用或处理响应时出错: {str(e)}")
                import traceback
                print(traceback.format_exc())
                # 保存错误信息
                error_path = os.path.join(OUTPUT_DIR, f"{base_name}_error.txt")
                with open(error_path, 'w', encoding='utf-8') as f:
                    f.write(f"处理Prompt {prompt_num}时出错: {str(e)}\n\n")
                    f.write(traceback.format_exc())
                print(f"错误信息已保存到: {error_path}")
            
            # 休息一下，避免API限制
            print("等待5秒后处理下一个prompt...")
            time.sleep(5)
            
        except Exception as e:
            print(f"处理Prompt {prompt_num}时发生错误: {str(e)}")
            import traceback
            print(traceback.format_exc())
    
    print("\n所有prompt处理完成!")

# 执行主函数
if __name__ == "__main__":
    # 确保输出目录存在
    os.makedirs(OUTPUT_DIR, exist_ok=True)
    
    # 开始处理
    process_each_prompt()

txt_file_1: 生成结果/defect_analysis\致欧-2025-01-10之后VOC数据_缺陷象限分析_完整列表.txt
txt_file_2: 生成结果/needs_analysis\致欧-2025-01-10之后VOC数据_用户需求象限分析_完整列表.txt
txt_file_3: 生成结果/matches_analysis\致欧-2025-01-10之后VOC数据_功能场景匹配象限分析_完整列表.txt
读取所有数据文件...
✓ 已读取: excel_file_1
✓ 已读取: excel_file_2
✓ 已读取: excel_file_3
✓ 已读取: excel_file_4


✓ 已读取: excel_file_5
✓ 已读取: excel_file_6
✓ 已读取: excel_file_7
✓ 已读取: excel_file_8
✓ 已读取: md_file_1
✓ 已读取: txt_file_1
✓ 已读取: txt_file_2
✓ 已读取: txt_file_3

所有数据文件读取完成，共 12 个文件


处理 Prompt 1: 建议表1-表达-流量关系表Prompt.txt
输出文件: 建议表1-表达-流量关系表.xlsx
Prompt长度: 2165 字符
调用API中...

====================思考过程====================



好的，我现在需要

根据用户提供的prompt和数据

生成一个符合要求的Excel

...

>
    </tbody

>
</table>


```



API调用完成，耗时: 44.36秒
已保存思考过程到: 生成结果/last_quadrant\建议表6-改进优先级排序_reasoning.txt
已保存原始回答到: 生成结果/last_quadrant\建议表6-改进优先级排序_result.txt
处理API响应并生成Excel...
成功将1个表格保存到Excel: 生成结果/last_quadrant\建议表6-改进优先级排序.xlsx
✓ 成功生成Excel文件: 生成结果/last_quadrant\建议表6-改进优先级排序.xlsx
等待5秒后处理下一个prompt...



所有prompt处理完成!
